In [2]:
import torch
import math
import random
import numpy as np

In [3]:
# autograd globally off
torch.set_grad_enabled(False)

In [4]:
class Module (object):
    def __init__(self):
        self.parameters = []
    def forward (self , * input ):
        raise NotImplementedError
    def backward (self , * gradwrtoutput ):
        raise NotImplementedError
    def param (self):
        return []

In [5]:
def generate_data(num):
#   using pytorch tensor operation
#   return the generated random points and labels
    random_points = torch.rand((num, 2))
    # check if the point is in the circle centered at (0.5,0.5) of radius 1/sqrt(2*math.pi)
    return random_points,torch.Tensor([int(pow(x[0]-0.5,2)+pow(x[1]-0.5,2)<1/(2*math.pi)) for x in random_points])

In [6]:
# def generate_data(num):
# #   return the generated random points and labels
#     random_points=np.random.rand(num,2)
#     # check if the point is in the circle centered at (0.5,0.5) of radius 1/sqrt(2*math.pi)
#     return random_points,[int(pow(x[0]-0.5,2)+pow(x[1]-0.5,2)<1/(2*math.pi)) for x in random_points]

In [7]:
train_num, train_labels=generate_data(1000)
test_num, test_labels=generate_data(1000)

In [8]:
print(train_num.shape)
print(len(train_labels))
print(test_num.shape)
print(len(test_labels))

torch.Size([1000, 2])
1000
torch.Size([1000, 2])
1000


In [9]:
class Linear(Module):
    
    def __init__(self, in_dim, out_dim, bias=True):
        super(Linear,self).__init__()
        init_range = 1. / math.sqrt(in_dim)
        self.weights = torch.Tensor(in_dim,  out_dim).uniform_(-init_range, init_range)
        self.grad_w = torch.zeros((in_dim,  out_dim))
        self.parameters = [(self.weights, self.grad_w)]
        if bias:
            # default bias initialization
            self.bias = torch.Tensor( out_dim).uniform_(-init_range, init_range)
            self.grad_b = torch.zeros( out_dim)
            self.parameters.append((self.bias, self.grad_b))
        else:
            self.bias = None

    def forward(self, input_):
        self.input = input_
        if self.bias is not None:
            return torch.addmm(self.bias, input_, self.weights)
        else:
            return input_.matmul(self.weights)

    def backward(self, grad_output):
        self.grad_w += self.input.t().matmul(grad_output)
        grad_input = grad_output.matmul(self.weights.t())
        if self.bias is not None:
            self.grad_b += grad_output.sum(dim=0)
        return grad_input
    
class ReLU(Module):

    def forward(self, input_):
        self.input = input_
        return torch.relu(input_)

    def backward(self, grad_output):
        return torch.mul((self.input > 0).int(), grad_output)
    
class Tanh(Module):

    def forward(self, input_):
        self.input = input_
        return torch.tanh(input_)

    def backward(self, grad_output):
        return torch.tanh(self.input).pow(2).mul(-1).add(1).mul(grad_output)

class Sequential(Module):

    def __init__(self, *args):
        super(Sequential,self).__init__()
        self.layers = list(args)
        for module in args:
            self.parameters += module.parameters

    def forward(self, input_):
        x = input_
        for layer in self.layers:
            x = layer.forward(x)
        return x

    def backward(self, loss_grad):
        y = loss_grad
        for layer in reversed(self.layers):
            y = layer.backward(y)
        return

class MSELoss(Module):
    def __init__(self):
        super(MSELoss,self).__init__()

    def __call__(self, input_, target):
        return self.forward(input_, target)

    def forward(self, input_, target):
        if input_.size() != target.size():
            raise Exception("Dimensions do not match")
        if target.dim() == 1:
            target = target.view(target.size(0), 1)
        return (input_ - target).pow(2).mean().item()

    def backward(self, input_, target):

        if input_.size() != target.size():
            raise Exception("Dimensions do not match")
        if target.dim() == 1:
            target = target.view(target.size(0), 1)

        return (input_ - target).mul(2).div(target.size(0))

class SGD:
    def __init__(self, parameters, learning_rate):
        self.parameters = parameters
        self.learning_rate = learning_rate

    def step(self):
        for w, dw in self.parameters:
            w.sub_(dw.mul(self.learning_rate))
            
class Model:
    def __init__(self, layers):
        self.layers = layers
        self.parameters = []
        for layer in layers:
            self.parameters += layer.parameters

    def __call__(self, input_):
        return self.forward(input_)

    def zero_grad(self):
        for w, dw in self.parameters:
            dw.zero_()

    def forward(self, input_):
        x = input_
        for layer in self.layers:
            x = layer.forward(x)
        return x

    def backward(self, loss_grad):
        y = loss_grad
        for layer in reversed(self.layers):
            y = layer.backward(y)
        return

In [16]:
def train_model(model, train_input, train_target, batch_size=100, n_epochs=250, loss=MSELoss(), learning_rate=0.1, print_loss=True):
    sample_size = train_input.size(0)
    sgd = SGD(model.parameters, learning_rate)
    for epoch in range(n_epochs):
        cumulative_loss = 0
        for n_start in range(0, sample_size, batch_size):
            # resetting the gradients
            model.zero_grad()
            output = model(train_input[n_start : n_start + batch_size])
            # accumulating the loss over the mini-batches
            cumulative_loss += loss(output, train_target[n_start : n_start + batch_size]) * batch_size
            # calculating the gradient of the loss wrt final outputs
            loss_grad = loss.backward(output, train_target[n_start : n_start + batch_size])
            # propagating it backward
            model.backward(loss_grad)
            # updating the parameters
            sgd.step()
        if print_loss:
            print("Epoch: %i" % epoch)
            print("Loss: %f" % (cumulative_loss / sample_size))


def accuracy(true_target, predicted):
    return true_target.argmax(dim=1).sub(predicted.argmax(dim=1)).eq(0).float().mean().item()

def generatePoint(n=1000):
    points = torch.Tensor(n, 2).uniform_(0, 1)
    label_1 = points.sub(0.5).pow(2).sum(axis=1).sub(1 / (math.pi * 2)).sign().view(-1, 1)
    label = torch.cat((label_1.mul(-1), label_1), 1).add(1).div(2)
    return points,label

In [17]:
layers = [Linear(2, 25), ReLU(), Linear(25, 25), ReLU(), Linear(25, 25),ReLU(), Linear(25, 2)]
model = Model(layers)

Trainpoint, Trainlabel =   generatePoint(1000)
Testpoint, Testlabel =  generatePoint(1000)  

train_model(model, Trainpoint, Trainlabel)
print("Train accuracy: %f" % accuracy(Trainlabel, model(Trainpoint)))
print("Test accuracy: %f" % accuracy(Testlabel, model(Testpoint)))

Epoch: 0
Loss: 0.298046
Epoch: 1
Loss: 0.250541
Epoch: 2
Loss: 0.249731
Epoch: 3
Loss: 0.248935
Epoch: 4
Loss: 0.248089
Epoch: 5
Loss: 0.247141
Epoch: 6
Loss: 0.246093
Epoch: 7
Loss: 0.244880
Epoch: 8
Loss: 0.243469
Epoch: 9
Loss: 0.241788
Epoch: 10
Loss: 0.239726
Epoch: 11
Loss: 0.237183
Epoch: 12
Loss: 0.233983
Epoch: 13
Loss: 0.230186
Epoch: 14
Loss: 0.225823
Epoch: 15
Loss: 0.222139
Epoch: 16
Loss: 0.225495
Epoch: 17
Loss: 0.228516
Epoch: 18
Loss: 0.225030
Epoch: 19
Loss: 0.222630
Epoch: 20
Loss: 0.220350
Epoch: 21
Loss: 0.215658
Epoch: 22
Loss: 0.213108
Epoch: 23
Loss: 0.209340
Epoch: 24
Loss: 0.206272
Epoch: 25
Loss: 0.201398
Epoch: 26
Loss: 0.196302
Epoch: 27
Loss: 0.192406
Epoch: 28
Loss: 0.187941
Epoch: 29
Loss: 0.183071
Epoch: 30
Loss: 0.175937
Epoch: 31
Loss: 0.170340
Epoch: 32
Loss: 0.163889
Epoch: 33
Loss: 0.158675
Epoch: 34
Loss: 0.152433
Epoch: 35
Loss: 0.146162
Epoch: 36
Loss: 0.140504
Epoch: 37
Loss: 0.135842
Epoch: 38
Loss: 0.130649
Epoch: 39
Loss: 0.122507
Epoch: 40
